**how to execute the program (in Jupyter Lab)**

esc, ctrl+a, ctrl+enter to run all cells

# Functions

## import of packages

In [52]:
import numpy as np # better arrays than inbuilt arrays
# import matplotlib.pyplot as plt # to plot stuff

import pandas as pd #for DataFrame tables
from IPython.display import display #to display dfs more nicely. works similar to head(), but is more flexible in how many columns/rows are shown
pd.set_option("display.max_rows", None) #to let display show full df
pd.set_option("display.max_columns", None)
from sklearn import preprocessing

import scipy.stats
#from scipy.stats import norm
import statistics

import math
import time #to measure runtime

## Expectation-Maximization    

In [53]:
def update_p_zj_given_xi(p_xi_given_zj, p_zj, p_zj_given_xi):
    ## both for PEPTIDE and PROTEIN inference. originally I had thought I could simplify this step for protein inference, but I don't understand why it doesn't work. so the protein inference step uses the same calculcation as the peptide inference step
    if inference_mode == "peptide" or "protein":
        numerator = p_xi_given_zj * p_zj # = ARRAY with same shape as P(Xi|Zj)
        denominator = np.expand_dims(np.sum(p_xi_given_zj * p_zj, axis=1), axis=-1) # = VECTOR with length of p Xi. expand_dims is necessary because else denominator has the shape (number of reads Xi,) instead of (number of reads Xi, 1). The entries of the vector are identical. But it does not have the same matrix "rank", which is why broadcasting fails without that 
        p_zj_given_xi = np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0) # if denominator = 0, return 0. This happens sometimes when the likelihood for Zj is equal to 0. This could alternatively be avoided by cropping the rows where the input P(Xi|Zj) scores equal to 0
        p_zj_given_xi = np.transpose(p_zj_given_xi)
       
    ## UNUSED. PLACEHOLDER for PROTEIN inference (denominator is has no loop). See inference_mode comment above for explanation why its unused.
    elif inference_mode == "PLACEHOLDER protein":
        p_zj_given_xi = (p_xi_given_zj * p_zj)/np.expand_dims(p_xi, axis=-1)
        p_zj_given_xi = np.transpose(p_zj_given_xi)
        
    else:
        print("ERROR: inference_mode was not set to peptide or protein")
        
    return p_zj_given_xi

In [54]:
def update_p_zj(p_zj, p_zj_given_xi):  
    if inference_mode == "peptide":
        p_zj = p_zj_given_xi.sum(axis=1)#/p_xi_given_zj.shape[1]

        p_zj_sum = np.sum(p_zj, axis = 0)
        p_zj = p_zj/p_zj_sum #so that p_zj is a fraction (i. e. a number between 0 and 1) 

    elif inference_mode == "protein":
        p_zj.fill(0) # reset all values of pzj to 0
        
        p_zj = np.sum(p_zj_given_xi * p_xi, axis = 1)
        p_zj_sum = np.sum(p_zj, axis = 0)
        p_zj = p_zj/p_zj_sum #so that p_zj is a fraction (i. e. a number between 0 and 1) 
    
    else:
        print("ERROR: inference_mode was not set to peptide or protein")
    
    return p_zj

In [55]:
def EM_convergence_checker(p_zj_old, p_zj, EM_convergence_minimum):
    difference_abs = abs(np.sum(p_zj_old[round(0.5*len(p_zj_old))::], axis=0, dtype=float) - np.sum(p_zj[round(0.5*len(p_zj_old))::], axis=0, dtype=float))
    if difference_abs > EM_convergence_minimum:
        return False
    
    else:
        return True   

In [56]:
def EM(p_xi_given_zj):
    global xi_len
    global zj_len
    xi_len = p_xi_given_zj.shape[0] # length of x (number of reads in pept infer, or peptides in prot infer)
    zj_len = p_xi_given_zj.shape[1] # length of z (number of peptides in pept infer, or proteins in prot infer)
    
    p_zj_given_xi = np.full((zj_len, xi_len), 0, dtype=float) #Initialisation based on array size of p_xi_given_zj -- same size, but transposed
    
    p_zj_initial = 1/zj_len #initial approximation: all zj equally likely, to jumpstart first iteration

    global p_zj
    p_zj = np.full(zj_len, p_zj_initial)
    p_zj_old = np.full(zj_len, 0)
    
    global EM_loopcounter 
    EM_loopcounter = 0
    EM_convergence_checker_result = False
        
    while (EM_convergence_checker_result == False) and EM_loopcounter < EM_loopcounter_max:
        p_zj_given_xi = update_p_zj_given_xi(p_xi_given_zj, p_zj, p_zj_given_xi)        
        p_zj_old = np.copy(p_zj, order='K', subok=False)
        p_zj = update_p_zj(p_zj, p_zj_given_xi)
        EM_convergence_checker_result = EM_convergence_checker(p_zj_old, p_zj, EM_convergence_minimum)

        EM_loopcounter = EM_loopcounter + 1
    return p_zj

## bootstrapping

In [57]:
def create_subarray_of_p_xi_given_zj(p_xi_given_zj):
    df_p_xi_given_zj = pd.DataFrame(p_xi_given_zj)
    df_p_xi_given_zj_sample = df_p_xi_given_zj.sample(frac=bootstrap_sampled_fraction, axis='rows', replace=True) # filters for a random partial dataset
    # display(pd.DataFrame(df_p_xi_given_zj_sample))
    p_xi_given_zj_subarray = df_p_xi_given_zj_sample.to_numpy()
    
    # print("p_xi_given_zj_subarray")
    # display(pd.DataFrame(p_xi_given_zj_subarray))
    
    return p_xi_given_zj_subarray

In [58]:
def bootstrap_EM():
    global p_zj_bootstrap_results_fraction
    global p_zj_peptide_copy
    global p_xi
    i = 0
    
    p_zj_bootstrap_results_absolute = np.full((p_xi_given_zj.shape[1]), 0, dtype=float)
      
    if bootstrap_sampled_fraction == -1: # no bootstrapping, i. e. use full dataset for EM. Since the EM is deterministic, there is no point in running multiple bootstrap runs
        if inference_mode == "peptide": 
            p_xi_given_zj_subarray = p_xi_given_zj
            p_zj_bootstrap_results_absolute = EM(p_xi_given_zj_subarray)

            p_zj_bootstrap_results_fraction = p_zj_bootstrap_results_absolute/np.sum(p_zj_bootstrap_results_absolute, axis = 0)
            print("Peptide Zj values from every bootstrap run (columns: peptides, rows: bootstrap run, displayed as fractions: ")
    
        elif inference_mode == "protein": #only difference: uses p(zj) values from peptide inferrence as the p_xi values. Also, update_p_zj() called by EM() uses a different formula than in pep infer
            p_xi = np.copy(p_zj_bootstrap_results_fraction_avg) # input is just a vector, because no bootstrapping.

            p_xi_given_zj_subarray = p_xi_given_zj
            p_zj_bootstrap_results_absolute = EM(p_xi_given_zj_subarray)

            p_zj_bootstrap_results_fraction = p_zj_bootstrap_results_absolute/np.sum(p_zj_bootstrap_results_absolute, axis = 0)
            print("PRE-CORRECTION: Protein Zj values from every bootstrap run (columns: protein, rows: bootstrap run, displayed as fractions: ")
    
    elif bootstrap_sampled_fraction != -1: # with bootstrapping, i. e. a subarray of the dataset is created for each bootstrap run
        if inference_mode == "peptide":
            while i < n_bootstrap_runs:
                p_xi_given_zj_subarray = create_subarray_of_p_xi_given_zj(p_xi_given_zj) 
                if i == 0:
                    p_zj_bootstrap_results_absolute = EM(p_xi_given_zj_subarray)
                else:
                    p_zj_bootstrap_results_absolute = np.vstack((p_zj_bootstrap_results_absolute, EM(p_xi_given_zj_subarray)))

                print("Bootstrap run #", i, ". EM loops: ", EM_loopcounter, sep="")
                if EM_loopcounter == EM_loopcounter_max:
                    print("WARNING: EM_loopcounter was reached, convergence likely still has not been reached. Consider increasing the maximum number of EM loops.")

                i = i + 1
            p_zj_bootstrap_results_fraction = p_zj_bootstrap_results_absolute/np.sum(p_zj_bootstrap_results_absolute, axis = 1)[0]    

        elif inference_mode == "protein":
            p_zj_peptide_copy = np.copy(p_zj_bootstrap_results_fraction) # input: an array

            while i < n_bootstrap_runs:
                p_xi_given_zj_subarray = p_xi_given_zj
                p_xi = p_zj_peptide_copy[i]
                
                if i == 0:
                    p_zj_bootstrap_results_absolute = EM(p_xi_given_zj_subarray)
                else:
                    p_zj_bootstrap_results_absolute = np.vstack((p_zj_bootstrap_results_absolute, EM(p_xi_given_zj_subarray)))

                print("Bootstrap run #", i, ". EM loops: ", EM_loopcounter, sep="")
                if EM_loopcounter == EM_loopcounter_max:
                    print("WARNING: EM_loopcounter was reached, convergence likely still has not been reached. Consider increasing the maximum number of EM loops.")

                i = i + 1
                
            p_zj_bootstrap_results_fraction = p_zj_bootstrap_results_absolute/np.sum(p_zj_bootstrap_results_absolute, axis = 1)[0]

In [59]:
def bootstrap_EM_analytics_AVG():
    global p_zj_bootstrap_results_fraction_avg
    
    if bootstrap_sampled_fraction == -1:
        p_zj_bootstrap_results_fraction_avg = np.copy(p_zj_bootstrap_results_fraction,order='K')
    elif bootstrap_sampled_fraction != -1:
        p_zj_bootstrap_results_fraction_avg = np.sum(p_zj_bootstrap_results_fraction, axis = 0)/p_zj_bootstrap_results_fraction.shape[0]

In [60]:
def bootstrap_EM_analytics_CI():
    outlier_on_each_side = (100-CI_percent)/(2*100) # i.e. 2.5% missing on each side if 95% CI
    
    lowindex = round(outlier_on_each_side*n_bootstrap_runs)
    highindex = round((1-outlier_on_each_side)*n_bootstrap_runs)
    
    bootstrap_CI_max = np.sort(p_zj_bootstrap_results_fraction, axis=0, kind="quicksort", order=None)[lowindex:highindex:][-1] #fetches last row from sorted table, ie the highest values. axis=0: sorting along each column
    bootstrap_CI_min = np.sort(p_zj_bootstrap_results_fraction, axis=0, kind="quicksort", order=None)[lowindex:highindex:][0] #same but for first row, highest values
    bootstrap_CI_minmax = np.stack((bootstrap_CI_min, bootstrap_CI_max))
    
    ## show full sorted table
    # global p_zj_bootstrap_results_fraction_CI
    # print("The", CI_percent, "% confidence interval of all zj bootstrapping values sorted, displayed as fractions: ")    
    # p_zj_bootstrap_results_fraction_CI = np.sort(p_zj_bootstrap_results_fraction, axis=0, kind="quicksort", order=None)[lowindex:highindex:] #axis=sorting along each column
    # display(pd.DataFrame(p_zj_bootstrap_results_fraction_CI))
    
    return bootstrap_CI_minmax

## output

In [61]:
def output():
    global p_zj_bootstrap_results_fraction_avg
    bootstrap_EM_analytics_AVG()
    bootstrap_CI_minmax = bootstrap_EM_analytics_CI()

    ### STDs (alternative to CI)
    p_zj_bootstrap_results_fraction_std = np.std(p_zj_bootstrap_results_fraction, axis = 0)
    
    if inference_mode == "protein":
        corrfactor = dyeseq_in_prot_match_count*np.sum(p_zj_bootstrap_results_fraction_avg/dyeseq_in_prot_match_count)

        p_zj_bootstrap_results_fraction_avg = p_zj_bootstrap_results_fraction_avg/corrfactor        
        if bootstrap_sampled_fraction != -1:
            p_zj_bootstrap_results_fraction_std = p_zj_bootstrap_results_fraction_std/corrfactor
            bootstrap_CI_minmax = bootstrap_CI_minmax/corrfactor
            
    # Combines AVG zj values (from all bootstrap runs), the STD, and the two bounds for the user-chosen confidence interval in one  
    if bootstrap_sampled_fraction == -1: #ie if no bootstrapping. Fills -CI and +CI columns with "N/A" values
        p_zj_avg_plus_CI = np.stack((p_zj_bootstrap_results_fraction_avg*100, np.full((len(p_zj_bootstrap_results_fraction_avg)), fill_value="N/A", dtype=None), np.full((len(p_zj_bootstrap_results_fraction_avg)), fill_value="N/A", dtype=None), np.full((len(p_zj_bootstrap_results_fraction_avg)), fill_value="N/A", dtype=None)), axis=1)    
        display(pd.DataFrame(p_zj_avg_plus_CI, columns = ["AVG [%]", "±STD [%]", "-CI [%]", "+CI [%]"]).astype({"AVG [%]":float}).round(4))
    else:
        p_zj_avg_plus_CI = np.stack((p_zj_bootstrap_results_fraction_avg, p_zj_bootstrap_results_fraction_std, bootstrap_CI_minmax[0], bootstrap_CI_minmax[1]), axis = 1)
        display(pd.DataFrame(p_zj_avg_plus_CI, columns = ["AVG [%]", "±STD [%]", "-CI [%]", "+CI [%]"]).round(4)*100)

    ### print user-settings
    print('\033[1m' + 'Settings of this run' + '\033[0m')
    print("Inference mode:", inference_mode)
    print("EM_convergence_minimum:", EM_convergence_minimum)
    print("EM_loopcounter_max:", EM_loopcounter_max)
    print("bootstrap_sampled_fraction:", bootstrap_sampled_fraction)
    print("n_bootstrap_runs:", n_bootstrap_runs)
    print("CI_percent:", CI_percent, "\n")

    ### print other
    print("Runtime: %s seconds ---" % round((time.time() - start_time),2)) #run time

# call functions

In [62]:
# esc, ctrl+a, ctrl+enter to run all cells

EM_convergence_minimum = float(input("EM_convergence_minimum? If nothing is entered, it is set to 0.0001.") or "0.0001")
EM_loopcounter_max = int(input("Maximum number of EM runs (per bootstrap run)? If nothing is entered, it is set to 200.") or "200") 
bootstrap_sampled_fraction = float(input("Fraction of subarray sampled for each bootstrap run? If nothing is entered, it is set to 0.8. If -1 is entered, bootstrapping is turned off. Note: Bootstrapping is always turned OFF for the protein inference part.") or "0.8")                   
n_bootstrap_runs = int(input("Number of bootstrap runs? If nothing is entered, it is set to 200.") or "200") 
CI_percent = float(input("Condidence interval? If nothing is entered, it is set to 95.") or "95")

EM_convergence_minimum? If nothing is entered, it is set to 0.0001. 
Maximum number of EM runs (per bootstrap run)? If nothing is entered, it is set to 200. 10
Fraction of subarray sampled for each bootstrap run? If nothing is entered, it is set to 0.8. If -1 is entered, bootstrapping is turned off. Note: Bootstrapping is always turned OFF for the protein inference part. 
Number of bootstrap runs? If nothing is entered, it is set to 200. 10
Condidence interval? If nothing is entered, it is set to 95. 


In [ ]:
inference_mode = "peptide"

p_xi_given_zj = np.genfromtxt("110 pept 10 prot set/1_p_xi_given_zj_pept_infer_input.csv", delimiter=',')

start_time = time.time() # to start measuring runtime
bootstrap_EM()
output()

In [ ]:
inference_mode = "protein"

dyeseq_in_prot_match_count = np.genfromtxt("110 pept 10 prot set/2_dyeseq_in_prot_match_count.csv", delimiter=',') # To calculate this the target proteome needs to be virtually digested and labelled (i.e. turned into dye sequences). Next, the reads' dye sequences are matched against this, and the number of matches per protein is counted. I did not write any code to solve this particular problem, but it seems like it should be fairly easy
p_xi_given_zj = np.genfromtxt("110 pept 10 prot set/2_p_xi_given_zj_prot_infer_input.csv", delimiter=',') # The likelihood for any dye seq Xi given a protein Zj is equal to the number of matches of that dye seq in a particular protein, divided by the sum of all dye seq matches of that protein. I did not write any code to solve this problem either

start_time = time.time() # to start measuring runtime
bootstrap_EM()
output()